In [1]:
# audio_dataset.ipynb 에서 만든 데이터셋 불러오기

import datasets

minds = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/dataset_01")

In [2]:
minds

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 5501
})

In [3]:
minds['intent_class'][:2]

[1, 1]

In [4]:
label2id = {'angry':'1','neutral':'0'}
id2label = {'1':'angry','0':'neutral'}
num_labels = len(id2label)

In [5]:
minds = minds.train_test_split(test_size=0.2)
minds

DatasetDict({
    train: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 4400
    })
    test: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 1101
    })
})

In [6]:
minds["train"][0]

{'audio': {'array': [0.0025634765625,
   0.002410888671875,
   0.002410888671875,
   0.001617431640625,
   0.0009765625,
   -0.001007080078125,
   -0.000732421875,
   -0.001617431640625,
   -0.00299072265625,
   -0.003692626953125,
   -0.004241943359375,
   -0.00433349609375,
   -0.004974365234375,
   -0.005401611328125,
   -0.005279541015625,
   -0.006103515625,
   -0.005706787109375,
   -0.006317138671875,
   -0.0068359375,
   -0.00726318359375,
   -0.008026123046875,
   -0.007598876953125,
   -0.007904052734375,
   -0.0086669921875,
   -0.00958251953125,
   -0.0089111328125,
   -0.009765625,
   -0.0093994140625,
   -0.009735107421875,
   -0.01007080078125,
   -0.010040283203125,
   -0.009857177734375,
   -0.009429931640625,
   -0.008026123046875,
   -0.0086669921875,
   -0.007720947265625,
   -0.007904052734375,
   -0.00714111328125,
   -0.006317138671875,
   -0.005889892578125,
   -0.00445556640625,
   -0.00323486328125,
   -0.002105712890625,
   -0.002288818359375,
   -0.001068115

In [7]:
id2label[str(0)]

'neutral'

In [8]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [9]:
from datasets import Audio

minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [10]:
minds

DatasetDict({
    train: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 4400
    })
    test: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 1101
    })
})

In [11]:
minds["train"][0]

{'audio': {'path': None,
  'array': array([ 2.56347656e-03,  2.41088867e-03,  2.41088867e-03, ...,
          0.00000000e+00, -9.15527344e-05, -3.05175781e-05]),
  'sampling_rate': 16000},
 'intent_class': 0}

In [12]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [13]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [16]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'projector.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./model/model-1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    # push_to_hub=True,
    # no_cuda =True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch,Training Loss,Validation Loss,Accuracy
0,0.495800,0.485899,0.782925
2,0.282500,0.302627,0.889192
2,0.280600,0.245260,0.900999
4,0.353900,0.293701,0.877384
4,0.196900,0.304674,0.908265
6,0.198700,0.255883,0.913715
6,0.137500,0.202036,0.923706
8,0.178400,0.221593,0.923706
8,0.191800,0.255922,0.899183
10,0.184500,0.200359,0.931880
